In [1]:
from scrap import Scraper
from RAG_QnA import RAG_Model
scrp = Scraper()
rag = RAG_Model()

2024-08-05 14:32:08.239621: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-05 14:32:08.263601: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-05 14:32:08.271055: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-05 14:32:10.481104: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /home/junaid-ul-hassan/.cache/huggingface/token
Login successful


In [19]:
url = "https://docs.portkey.ai/docs/guides/integrations/introduction-to-gpt-4o"

In [20]:
response = scrp.scrape_website(url=url)
if response==403:
    print("This Website don't allow scrapping")

In [21]:
response

200

In [22]:
rag.load_Database()

Its run..


In [26]:
rag.window_mem.clear()

In [28]:
print(rag.generateResponse("What is GPT-4o?"))

GPT-4o is an advanced AI model that handles a combination of text, audio, and video inputs and generates outputs in text, audio, and image formats. It supports text and image inputs currently, with additional modalities like audio being added soon. The model has improved function calling capabilities and can interpret and execute multiple commands in a more structured and efficient manner.</s>


In [25]:
print(rag.window_mem.buffer_as_str)

Human: What is langchain?
AI:  Langchain appears to be a platform that includes several components such as GPT-4o, Portkey, Supabase pgvector, and OpenAI SDK among others. It seems to be focused on building applications that utilize various models with different modalities like visual, audio, or text inputs. However, without more specific information about Langchain, it's hard to provide a more detailed explanation.</s>
